<a href="https://colab.research.google.com/github/Shakpro10/Machine-Learning-Projects/blob/main/tree/main/Deep%2520Learning/Classifying_coloured_images_of_cats_and_dogs_using_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import psutil
ram = psutil.virtual_memory()
print(f"Total RAM: {ram.total / (1024 ** 3):.2f} GB")


Total RAM: 334.56 GB


In [2]:
import tensorflow as tf

try:
    tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('TPU detected:', tpu_resolver.master())
    # List the available TPU workers
    workers = tpu_resolver.cluster_spec().as_dict().get('worker', [])
    print('TPU workers:', workers)
    if not workers:
        print("No TPU workers found.")
except ValueError:
    print('TPU not available.')

TPU detected: 
TPU workers: []
No TPU workers found.


In [3]:
import tensorflow as tf

try:
    tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu_resolver)
    tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
    strategy = tf.distribute.TPUStrategy(tpu_resolver)
    print("TPU initialized successfully.")
except ValueError as e:
    print('Error initializing TPU:', e)

TPU initialized successfully.


## **PROJECT - Classifying Colored Images of Cats and Dogs**

#### Details
- Convert the dimensions of the images to 28 x 28 pixels.
- The dataset consists of:
  - 2000 training images (1000 cats and 1000 dogs)
  - 1000 validation images (500 cats and 500 dogs)
  - 1000 test images (500 cats and 500 dogs)
- We will use data augmentation to improve model performance through various transformations.
- Further improvements in model performance will be achieved using transfer earning.


In [4]:
# Import the relevant libraries
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

In [5]:
import tensorflow as tf
from tensorflow import keras

In [6]:
# Mount drive to import dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Create a folder path
folder_path = '/content/drive/My Drive/ML_Dataset/cats_and_dogs_small/'
os.listdir(folder_path)  # Lists all files in the folder

['test', 'train', 'validation']

In [8]:
# Read and load the data from their various directories
train_dir = '/content/drive/My Drive/ML_Dataset/cats_and_dogs_small/train'
validation_dir = '/content/drive/My Drive/ML_Dataset/cats_and_dogs_small/validation'
test_dir = '/content/drive/My Drive/ML_Dataset/cats_and_dogs_small/test'


### Data Preprocessing

#### Steps:
- Decode the jpeg images to rgb pixels grids
- Convert the grids into floating point tensors
- Rescale the pixel values to (0, 1) interval

In [9]:
# Import the library to perform all the preprocessing steps
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
# Generating batches of tensor image data, rescale, set target size and batch size

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [11]:
# Import VGGNet architecture for transfer learning

from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

58889256/58889256 [==============================] - 0s 0us/step


In [12]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [13]:
# Import the libraries for training the neural network
from tensorflow.keras import layers, models, optimizers

In [14]:
# Arranging the layers in sequence

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Compiling
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(learning_rate=2e-5),
              metrics=['accuracy'])

# Use actual input data (dummy data) to build the model
dummy_data = np.random.random((20, 150, 150, 3))  # Use a batch of 20 samples with input shape (150, 150, 3)
model.predict(dummy_data)  # Perform a forward pass to build the model

# Now, print the layer names and output shapes using the `.output` property
for layer in model.layers:
    print(f"Layer: {layer.name}, Output Tensor: {layer.output}")  # This should work without error

# Print the model summary
model.summary()

1/1 [==============================] - 0s 430ms/step
Layer: vgg16, Output Tensor: KerasTensor(type_spec=TensorSpec(shape=(None, 4, 4, 512), dtype=tf.float32, name=None), name='block5_pool/MaxPool:0', description="created by layer 'block5_pool'")
Layer: flatten, Output Tensor: KerasTensor(type_spec=TensorSpec(shape=(None, 8192), dtype=tf.float32, name=None), name='flatten/Reshape:0', description="created by layer 'flatten'")
Layer: dense, Output Tensor: KerasTensor(type_spec=TensorSpec(shape=(None, 256), dtype=tf.float32, name=None), name='dense/Relu:0', description="created by layer 'dense'")
Layer: dense_1, Output Tensor: KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), name='dense_1/Sigmoid:0', description="created by layer 'dense_1'")
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
   

In [15]:
# Reload the model weights for the first epoch

model.load_weights('/content/drive/My Drive/ML_Dataset/Cat_and_dogs_model.weights.h5')

In [16]:
# Saving from the resumed training
checkpoint_cb = keras.callbacks.ModelCheckpoint('/content/drive/My Drive/ML_Dataset/Cat_and_dogs_model.keras', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)

In [17]:
val_data = next(validation_generator)
print(val_data[0].shape)  # Print shape of images
print(val_data[1].shape)  # Print shape of labels

(20, 150, 150, 3)
(20,)


In [18]:
# Fit the model

history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=50,
    callbacks=[checkpoint_cb, early_stopping_cb],
    initial_epoch=22)

Epoch 23/50
100/100 [==============================] - 669s 7s/step - loss: 0.0454 - accuracy: 0.9835 - val_loss: 0.0943 - val_accuracy: 0.9660
Epoch 24/50
100/100 [==============================] - 170s 2s/step - loss: 0.0321 - accuracy: 0.9875 - val_loss: 0.1144 - val_accuracy: 0.9540
Epoch 25/50
100/100 [==============================] - 169s 2s/step - loss: 0.0424 - accuracy: 0.9850 - val_loss: 0.1266 - val_accuracy: 0.9600
Epoch 26/50
100/100 [==============================] - 169s 2s/step - loss: 0.0276 - accuracy: 0.9905 - val_loss: 0.1117 - val_accuracy: 0.9680
Epoch 27/50
100/100 [==============================] - 168s 2s/step - loss: 0.0503 - accuracy: 0.9790 - val_loss: 0.1464 - val_accuracy: 0.9460
Epoch 28/50
100/100 [==============================] - 168s 2s/step - loss: 0.0274 - accuracy: 0.9905 - val_loss: 0.1060 - val_accuracy: 0.9660


In [19]:
# Store the accuracies and losses in a dataframe

hist_df = pd.DataFrame(history.history)
hist_df.tail()

,loss,accuracy,val_loss,val_accuracy
1,0.032101,0.9875,0.114442,0.954
2,0.042394,0.9850,0.126633,0.960
3,0.027645,0.9905,0.111717,0.968
4,0.050339,0.9790,0.146431,0.946
5,0.027370,0.9905,0.106003,0.966


In [20]:
# Save the accuracies and losses to a *csv file

hist_csv_file = os.path.join('/content/drive/My Drive/ML_Dataset', 'history.csv')
with open(hist_csv_file, mode='w', newline='') as f:
    hist_df.to_csv(f, index=False)

In [21]:
# Generating batches of tensor image data from the test data, rescale, set target size and batch size

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 1000 images belonging to 2 classes.


In [22]:
# Check for the model accuracy from the test set

model.evaluate(test_generator, steps=50)

50/50 [==============================] - 107s 2s/step - loss: 0.1179 - accuracy: 0.9610


[0.11786474287509918, 0.9610000252723694]